In [3]:
# ===== PIPELINE COMPLETO ULTRA OTIMIZADO PARA 80%+ =====
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from lightgbm import LGBMClassifier
import xgboost as xgb
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')

# ===== 1. CARREGAMENTO DOS DADOS =====
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_sub = pd.read_csv('sample_submission.csv')

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print(f"Target distribution:\n{train['labels'].value_counts(normalize=True)}")

# ===== 2. PRÉ-PROCESSAMENTO AVANÇADO =====
print("\n Iniciando pré-processamento avançado...")

age_cols = ['age_first_funding_year', 'age_last_funding_year',
           'age_first_milestone_year', 'age_last_milestone_year']

# Imputação inteligente de NaNs (sem warnings)
for col in age_cols:
    train[col] = train[col].fillna(train[col].median())
    test[col] = test[col].fillna(train[col].median())

train['funding_total_usd'] = train['funding_total_usd'].fillna(train['funding_total_usd'].median())
test['funding_total_usd'] = test['funding_total_usd'].fillna(train['funding_total_usd'].median())

# ===== 3. FEATURE ENGINEERING AVANÇADO =====
print("Criando features avançadas...")

# Features básicas (já testadas)
train['funding_per_round'] = train['funding_total_usd'] / (train['funding_rounds'] + 1e-6)
test['funding_per_round'] = test['funding_total_usd'] / (test['funding_rounds'] + 1e-6)

train['milestones_per_year'] = train['milestones'] / (train['age_last_milestone_year'] + 1e-6)
test['milestones_per_year'] = test['milestones'] / (test['age_last_milestone_year'] + 1e-6)

train['has_multiple_rounds'] = (train['funding_rounds'] > 1).astype(int)
test['has_multiple_rounds'] = (test['funding_rounds'] > 1).astype(int)

train['funding_age_span'] = train['age_last_funding_year'] - train['age_first_funding_year']
test['funding_age_span'] = test['age_last_funding_year'] - test['age_first_funding_year']

train['relationships_per_milestone'] = train['relationships'] / (train['milestones'] + 1e-6)
test['relationships_per_milestone'] = test['relationships'] / (test['milestones'] + 1e-6)

# Features extras para ganhar mais poder preditivo
train['funding_efficiency'] = train['funding_total_usd'] / (train['relationships'] + 1e-6)
test['funding_efficiency'] = test['funding_total_usd'] / (test['relationships'] + 1e-6)

train['milestone_density'] = train['milestones'] / (train['funding_rounds'] + 1e-6)
test['milestone_density'] = test['milestones'] / (test['funding_rounds'] + 1e-6)

train['avg_funding_per_participant'] = train['funding_total_usd'] / (train['avg_participants'] + 1e-6)
test['avg_funding_per_participant'] = test['funding_total_usd'] / (test['avg_participants'] + 1e-6)

# Features de interação (combinações que podem ser importantes)
train['funding_milestone_ratio'] = train['funding_total_usd'] / (train['milestones'] + 1e-6)
test['funding_milestone_ratio'] = test['funding_total_usd'] / (test['milestones'] + 1e-6)

# Tratar infinitos e NaNs
train.replace([np.inf, -np.inf], np.nan, inplace=True)
test.replace([np.inf, -np.inf], np.nan, inplace=True)
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

print(f"Features criadas! Train shape: {train.shape}")

# ===== 4. ENCODING E SCALING =====
print("Aplicando encoding e scaling...")

# One-Hot Encoding
train_encoded = pd.get_dummies(train, columns=['category_code'], prefix='cat')
test_encoded = pd.get_dummies(test, columns=['category_code'], prefix='cat')
train_encoded, test_encoded = train_encoded.align(test_encoded, join='left', axis=1, fill_value=0)

# Scaling das features numéricas
new_features = ['funding_per_round', 'milestones_per_year', 'has_multiple_rounds',
                'funding_age_span', 'relationships_per_milestone', 'funding_efficiency',
                'milestone_density', 'avg_funding_per_participant', 'funding_milestone_ratio']

numeric_cols = ['relationships', 'funding_rounds', 'funding_total_usd',
               'milestones', 'avg_participants'] + age_cols + new_features

scaler = StandardScaler()
train_encoded[numeric_cols] = scaler.fit_transform(train_encoded[numeric_cols])
test_encoded[numeric_cols] = scaler.transform(test_encoded[numeric_cols])

# Separar features e target
X = train_encoded.drop(['labels'], axis=1)
y = train_encoded['labels']
if 'labels' in test_encoded.columns:
    test_encoded = test_encoded.drop(['labels'], axis=1)
X_test = test_encoded

print(f"Dataset final: X shape = {X.shape}, X_test shape = {X_test.shape}")

# ===== 5. SPLIT ESTRATIFICADO =====
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,
                                                  random_state=42, stratify=y)

print(f"Split: Train={X_train.shape[0]}, Val={X_val.shape[0]}")

# ===== 6. BALANCEAMENTO COM SMOTE =====
print("\n Aplicando SMOTE para balanceamento...")
sm = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = sm.fit_resample(X_train, y_train)

print(f" Antes SMOTE: {y_train.value_counts().to_dict()}")
print(f" Depois SMOTE: {y_train_balanced.value_counts().to_dict()}")

# ===== 7. MODELOS INDIVIDUAIS OTIMIZADOS =====
print("\n Treinando modelos individuais otimizados...")

# Random Forest Ultra Otimizado
rf_model = RandomForestClassifier(
    n_estimators=800,         # Mais árvores para reduzir variância
    max_depth=25,             # Profundidade maior
    max_features='sqrt',      # Melhor para evitar overfitting
    min_samples_split=2,      # Permitir splits mais agressivos
    min_samples_leaf=1,       # Folhas menores
    bootstrap=True,           # Bagging
    random_state=42,
    n_jobs=-1,               # Usar todos os cores
    class_weight='balanced'   # Balanceamento adicional
)

# LightGBM Ultra Otimizado
lgbm_model = LGBMClassifier(
    n_estimators=1000,        # Mais árvores
    max_depth=25,             # Profundidade maior
    learning_rate=0.03,       # Taxa menor para melhor convergência
    subsample=0.8,            # Bagging
    colsample_bytree=0.8,     # Feature sampling
    reg_alpha=0.1,            # Regularização L1
    reg_lambda=0.1,           # Regularização L2
    class_weight='balanced',
    random_state=42,
    verbose=-1                # Silencioso
)

# XGBoost Ultra Otimizado
xgb_model = xgb.XGBClassifier(
    n_estimators=800,
    max_depth=8,
    learning_rate=0.03,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=0.1,
    random_state=42,
    scale_pos_weight=1,       # Balanceamento já feito pelo SMOTE
    eval_metric='logloss',
    use_label_encoder=False
)

print("Treinando Random Forest...")
rf_model.fit(X_train_balanced, y_train_balanced)

print("Treinando LightGBM...")
lgbm_model.fit(X_train_balanced, y_train_balanced)

print("Treinando XGBoost...")
xgb_model.fit(X_train_balanced, y_train_balanced)

print("Modelos individuais treinados!")

# ===== 8. STACKING CLASSIFIER (MAIS PODEROSO QUE VOTING) =====
print("\n Criando Stacking Classifier...")

# Meta-modelo para combinar as previsões
meta_model = LogisticRegression(
    max_iter=1000,
    random_state=42,
    class_weight='balanced'
)

# Stacking Classifier
stacking_model = StackingClassifier(
    estimators=[
        ('rf', rf_model),
        ('lgbm', lgbm_model),
        ('xgb', xgb_model)
    ],
    final_estimator=meta_model,
    passthrough=True,         # Inclui features originais no meta-modelo
    cv=5,                     # Cross-validation para treinar meta-modelo
    n_jobs=-1
)

print("Treinando Stacking Classifier...")
stacking_model.fit(X_train_balanced, y_train_balanced)

print("Stacking Classifier treinado!")

# ===== 9. THRESHOLD TUNING ULTRA FINO =====
print("\n THRESHOLD TUNING ULTRA FINO...")
print("=" * 50)

# Predições no conjunto de validação
y_prob_val = stacking_model.predict_proba(X_val)[:, 1]

# Threshold tuning com passo super fino (0.001)
best_acc = 0
best_thr = 0.5

print("Testando thresholds de 0.30 a 0.80 com passo 0.001...")
for thr in np.arange(0.30, 0.81, 0.001):
    preds_thr = (y_prob_val > thr).astype(int)
    acc = (preds_thr == y_val).mean()
    if acc > best_acc:
        best_acc = acc
        best_thr = thr

print(f"\n MELHOR THRESHOLD: {best_thr:.3f}")
print(f"MELHOR ACCURACY: {best_acc:.4f} ({best_acc*100:.1f}%)")

# Relatório detalhado com o melhor threshold
y_pred_final = (y_prob_val > best_thr).astype(int)
auc_final = roc_auc_score(y_val, y_prob_val)

print(f"\n RELATÓRIO FINAL (threshold = {best_thr:.3f}):")
print(f"AUC: {auc_final:.4f}")
print(classification_report(y_val, y_pred_final))

# ===== 10. MODELO FINAL E SUBMISSÃO =====
print("\n TREINANDO MODELO FINAL E GERANDO SUBMISSÃO...")

# Treinar stacking com todos os dados de treino (sem split)
final_stacking = StackingClassifier(
    estimators=[
        ('rf', rf_model),
        ('lgbm', lgbm_model),
        ('xgb', xgb_model)
    ],
    final_estimator=meta_model,
    passthrough=True,
    cv=5,
    n_jobs=-1
)

# Aplicar SMOTE em todo o dataset de treino
X_full_balanced, y_full_balanced = sm.fit_resample(X, y)

# Treinar modelo final
final_stacking.fit(X_full_balanced, y_full_balanced)

# Predições no test set com threshold otimizado
y_prob_test = final_stacking.predict_proba(X_test)[:, 1]
test_predictions = (y_prob_test > best_thr).astype(int)

# Criar submissão
submission = pd.DataFrame({
    'id': sample_sub['id'],
    'labels': test_predictions
})

submission.to_csv('my_submission_ultra_optimized.csv', index=False)

print("Submissão salva como 'my_submission_ultra_optimized.csv'")
print(f"Modelo usado: Stacking Classifier (RF + LGBM + XGB)")
print(f"Balanceamento: SMOTE")
print(f"Threshold otimizado: {best_thr:.3f}")
print(f"Acurácia alcançada: {best_acc:.1%}")
print(f"AUC: {auc_final:.4f}")

# Verificar distribuição das predições
print(f"\n Distribuição das predições no test set:")
print(f"Classe 0 (insucesso): {(test_predictions == 0).sum()} startups")
print(f"Classe 1 (sucesso): {(test_predictions == 1).sum()} startups")
print(f"Proporção de sucesso: {test_predictions.mean():.1%}")

print("\n" + "="*60)
print("RESUMO FINAL:")
print("="*60)
print(f"• Features criadas: {len(new_features)} novas variáveis")
print(f"• Balanceamento: SMOTE aplicado")
print(f"• Modelos: Random Forest + LightGBM + XGBoost")
print(f"• Meta-modelo: Stacking Classifier")
print(f"• Threshold: {best_thr:.3f} (otimizado com passo 0.001)")
print(f"• Acurácia final: {best_acc:.1%}")
print(f"• AUC: {auc_final:.4f}")
print("="*60)

Train shape: (646, 33)
Test shape: (277, 32)
Target distribution:
labels
1    0.647059
0    0.352941
Name: proportion, dtype: float64

 Iniciando pré-processamento avançado...
Criando features avançadas...
Features criadas! Train shape: (646, 42)
Aplicando encoding e scaling...
Dataset final: X shape = (646, 74), X_test shape = (277, 74)
Split: Train=516, Val=130

 Aplicando SMOTE para balanceamento...
 Antes SMOTE: {1: 334, 0: 182}
 Depois SMOTE: {0: 334, 1: 334}

 Treinando modelos individuais otimizados...
Treinando Random Forest...
Treinando LightGBM...
Treinando XGBoost...
Modelos individuais treinados!

 Criando Stacking Classifier...
Treinando Stacking Classifier...
Stacking Classifier treinado!

 THRESHOLD TUNING ULTRA FINO...
Testando thresholds de 0.30 a 0.80 com passo 0.001...

 MELHOR THRESHOLD: 0.300
MELHOR ACCURACY: 0.8000 (80.0%)

 RELATÓRIO FINAL (threshold = 0.300):
AUC: 0.8173
              precision    recall  f1-score   support

           0       0.81      0.57    